In [1]:
import pandas as pd
import numpy as np

from konlpy.tag import Mecab

In [2]:
mecab = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic") # dictation path 설정 / mecab-ko-dic이 mecab의 단어사전임

In [6]:
counseling = pd.read_csv('using_data/counseling_nolabel_4.csv', encoding = 'euc-kr')
cat_nouns_df = pd.read_csv('using_data/cluster_nouns.csv')

### SCORE

In [4]:
def vocab_dictionary_onetext(text_list): 
    vocab = list(set(sum([mecab.nouns(i) for i in text_list], [])))
    vocab1 = [i for i in vocab if len(i) >= 2]
    
    rm = ['어딜까', '가요', '건가요', '걸까요', '뭔가요', '입니다', '주세요', '취업', '지원', '어디','정책','이나','자리']
    for i in vocab1:
        if i in rm:
            vocab1.remove(i)
     
    add = ['돈', '일']
    for i in add:
        if i in vocab:
            vocab1.append(i)
        
    vocab1.sort()
    return vocab1

In [5]:
def tf(t, d):
    df = 0
    for word in mecab.nouns(d):
        if word == t:
            df += 1
    return df

def make_tf_dataset(text_list):
    N = len(text_list) # 총 문서의 수: 1
    result = []
    for i in range(N):
        result.append([])
        d = text_list[i]
        for j in range(len(vocab_dictionary_onetext(text_list))):
            t = vocab_dictionary_onetext(text_list)[j]        
            result[-1].append(tf(t, d))

    tf_ = pd.DataFrame(result, columns = vocab_dictionary_onetext(text_list))
    # tf_.sum(axis=0)
    return tf_

In [10]:
text_list = ['백수 생활한 지 오래되어서 일하기 싫고 귀찮아요. 일 안하고 먹고 살게 해주세요']
make_tf_dataset(text_list)

백수  생활  일
0   1   1  2

## 표준화한 점수 (저장 완료. 안돌려도됨)

In [11]:
from sklearn.preprocessing import StandardScaler

# Standardization 평균 0 / 분산 1
scaler = StandardScaler()   

scaler = scaler.fit_transform(cat_nouns_df[['cnt_0','cnt_1','cnt_2','cnt_3','cnt_4']])

In [12]:
cat_nouns_df[['cnt_0','cnt_1','cnt_2','cnt_3','cnt_4']] = pd.DataFrame(scaler)

In [13]:
# cat_nouns_df.to_csv('cluster_nouns.csv', index = False)

##  여기부터 돌리기

In [8]:
def cat_score(tf_):
    score = []

    for i in range(5):
        tf_trans = tf_.transpose().reset_index()
        tf_trans.columns = ['cat_{}_nouns'.format(i),'cnt_{}'.format(i)]

        temp_df = pd.merge(cat_nouns_df[['cat_{}_nouns'.format(i),'cnt_{}'.format(i)]], tf_trans, on = 'cat_{}_nouns'.format(i), how = 'inner')

        score.append(sum(temp_df['cnt_{}_x'.format(i)] * temp_df['cnt_{}_y'.format(i)]) )
    return score

In [ ]:
score = cat_score(make_tf_dataset(text_list))

In [62]:
def whatcategory(score_list):
    category = []
    for i in range(len(score_list)):
        if score_list[i] == max(score_list):
            if i == 0:
                category.append('edu')
            if i == 1:
                category.append('consult')
            if i == 2:
                category.append('money')
            if i == 3:
                category.append('interview')
            if i == 4:
                category.append('job')
    return category

In [79]:
category = whatcategory(score)

In [123]:
pd.options.display.max_colwidth = 40

In [360]:
idf_df = pd.read_csv('using_data/idf_정책_20211019.csv', encoding = 'euc-kr')
policy = pd.read_csv('using_data/content_tagging_2.csv', encoding = 'euc-kr')
similar = pd.read_csv('using_data/유사어.csv', encoding = 'euc-kr')

In [222]:
# similar = pd.merge(similar, idf_df, on = 'word', how = 'left')
# similar.to_csv('using_data/유사어.csv', encoding = 'euc-kr', index = False)

In [67]:
def df_for_recommend(category):
    using_df = policy.loc[policy['money'] + policy['interview'] + policy['edu'] + policy['job'] + policy['consult'] == 0] # 일단 어떤 카테고리에도 포함안되는 애들은 포함시키고 봄

    for i in category:
        if i == 'money':
            using_df = pd.concat([using_df, policy.loc[policy['money'] == 1]])
        if i == 'interview':
            using_df = pd.concat([using_df, policy.loc[policy['interview'] == 1]])
        if i == 'edu':
            using_df = pd.concat([using_df, policy.loc[policy['edu'] == 1]])
        if i == 'job':
            using_df = pd.concat([using_df, policy.loc[policy['job'] == 1]])
        if i == 'consult':
            using_df = pd.concat([using_df, policy.loc[policy['consult'] == 1]])
        
    using_df = using_df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False).reset_index(drop = True)
    return using_df

In [68]:
# using_df = policy # 전체 카테고리에서 추천할 때 얘 돌리기

In [86]:
def each_policy_nouns(using_df):
    nouns_list = []
    policy_index = []
    policy_nouns = []

    for i in range(len(using_df['content'])):
        nouns_list.extend(mecab.nouns(using_df['content'][i]))
        policy_index.append(using_df['index'][i])
        policy_nouns.append(mecab.nouns(using_df['content'][i]))

    policy_nouns = pd.DataFrame({'index': policy_index,
                                 'nouns': policy_nouns})
    return policy_nouns

In [311]:
text_list = ['예술 관련 취업하려고 하는데요. 비전공자여서요 혹시 국비 교육같은거 받을 수 있을까요?']
set(vocab_dictionary_onetext(text_list))

{'관련', '교육', '국비', '예술', '전공'}

In [ ]:
def each_policy_score(policy_nouns):
    score_list = []

    for i in policy_nouns['nouns']:
        
        text_words = set(vocab_dictionary_onetext(text_list))
        policy_words = set(i)
        
        intersection_words = list(text_words.intersection(policy_words))
    
        idf_df['cnt'] = pd.Series([i in intersection_words for i in idf_df['word']])
    
        score_list.append((idf_df['IDF'] * idf_df['cnt']).sum())
    return score_list

In [315]:
each_policy_nouns_df = each_policy_nouns(df_for_recommend(category))

In [390]:
policy_nouns = each_policy_nouns(df_for_recommend(category))

In [ ]:
each_policy_nouns(df_for_recommend(category))

In [1]:
###########################

def each_policy_score(policy_nouns):
    
    score_list = []

    for i, j in enumerate(policy_nouns['nouns']): # 각 정책들마다의 단어 리스트

        # 정책단어와 고민글단어의 교집합
        intersection_words = list(set(set(vocab_dictionary_onetext(text_list))).intersection(set(j)))

        inters_idf_df = idf_df.copy()
        inters_idf_df['cnt'] = pd.Series([i in intersection_words for i in inters_idf_df['word']])
        score1 = (inters_idf_df['IDF'] * inters_idf_df['cnt']).sum()

        # 상담단어 - 정책단어 차집합
        difference_words = list(set(vocab_dictionary_onetext(text_list)) - set(j))
        differ = []

        for i,j in enumerate(difference_words):
            if len(similar.loc[similar['similar'] == j]) > 0:
                differ.extend(list(similar.loc[similar['similar'] == j,'word']))

        intersection_words = list(set(set(differ)).intersection(set(j)))
        inters_idf_df = idf_df.copy()
        inters_idf_df['cnt'] = pd.Series([i in intersection_words for i in inters_idf_df['word']])
        score2 = (inters_idf_df['IDF'] * inters_idf_df['cnt']).sum()

        score_list.append(score1 + score2)
    return score_list

In [398]:
using_df = df_for_recommend(category)

In [401]:
using_df['score'] = score_list

In [404]:
pd.options.display.max_colwidth = 1000

In [305]:
# pd.set_option('display.max_rows', None)

In [200]:
tf_ = make_tf_dataset(text_list)

In [201]:
score = cat_score(tf_)

In [202]:
category = whatcategory(score)

In [203]:
using_df = df_for_recommend(category)

In [114]:
each_policy_nouns_df = each_policy_nouns(using_df)

In [197]:
using_df['score'] = score_list

In [146]:
def check_my_poilcy(text_list):
    tf_ = make_tf_dataset(text_list)
    score = cat_score(tf_)
    category = whatcategory(score)
    using_df = df_for_recommend(category)
    each_policy_nouns_df = each_policy_nouns(using_df)
    using_df['score'] = each_policy_score(each_policy_nouns_df)
    
    return using_df[['content','money','edu','interview','job','consult','score']].sort_values(by='score', ascending = False).iloc[0,0]

In [153]:
text_list = [input()]
check_my_poilcy(text_list)

채용사이트에 올린 저의 자소서와 이력서를 좀더 디테일하게 (경력서술이나 자격증 유무 관련) 수정하고 싶습니다.


'서울형 뉴딜 일자리청년을 비롯한 참여자들에게 일경험과 취업에 필요한 다양한 교육,서비스를 동시에 제공해 사업 참여 후 민간일자리로의 취업을 돕는 공공 일자리 사업 정책- 월 최대 임금 235만원 지급( 서울형 생활임금 시급 : 10,702원)\n- 최대 23개월간 안정적으로 근무가능(풀타임/파트타임 선택가능)\n- 일하는 동안 관련 경력을 쌓아 민간일자리 취업(2019년 참여자 취업률 52.9%)\n- 최대 130시간 취업교육,  자격증 응시비용 지원'